In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from konlpy.tag import Hannanum, Komoran, Kkma, Twitter
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import scipy.cluster.hierarchy as shc
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

In [18]:
news_df = pd.read_csv('news_df.csv', index_col=0)
news_df

,cate,news
0,eco,['YTN라디오(FM 94.5) [YTN\xa0뉴스FM\xa0슬기로운 라디오생활] ...
1,eco,['MG손보 부실 여파… KDB생명 매각도 ‘빨간불’실적 개선위해 공적자금 투입 3...
2,eco,['인건비·임대료·농산물 ‘급등’ 식음료 기업들 가격 인상 조짐서울 마포구에 거주하...
3,eco,['“1187만대 덜 팔고 78조7천억원 더 벌었다.”전 세계 대표 완성차 업체들 ...
4,eco,"[""“이제 오해를 푸는 일이 남았습니다. 학자로서 양심을 걸고 모다모다 샴푸를 보는..."
5,eco,['기사내용 요약 IC인사이트 분석결과…파운드리 제외 기준 [서울=뉴시스] 이인준 ...
6,eco,"[""(세종=뉴스1) 권혁준 기자 = 신종 코로나바이러스 감염증(코로나19)의 여파로..."
7,eco,['(세종=뉴스1) 서미선 기자 = 한국경제에 인플레이션 경고등이 켜지면서 국내외 ...
8,eco,['기사내용 요약 통계청\' 한국의 SDGs 이행보고서 2022\' 발간 중3·고2...
9,eco,"["" 코로나19 이후 중·고등학생의 주요 과목 기초학력 미달 비율이 크게 증가하는 ..."


In [19]:
def text_clean(doc):                    # 정규표현식을 이용하여 한글이 아닌것을 제외
    text = re.sub('[^가-핳ㄱ-ㅎㅏ-ㅣ ]','', doc)  
    return text

In [20]:
news_df['news'] = news_df.news.apply(lambda x : text_clean(x))

In [21]:
news_df

,cate,news
0,eco,라디오 뉴스슬기로운 라디오생 방송일시 년월 일 수요일 진이웅 아나운서 출연 박합수...
1,eco,손보 부실 여파 생명 매각도 빨간불실적 개선위 공적자금 투입 차례 매각 시도 번번이...
2,eco,인건비임대료농산물 급등 식음료 기업들 가격 인상 조짐서울 마포구에 거주하는 직장인 ...
3,eco,만대 덜 팔고 조천억원 더 벌었다전 세계 대표 완성차 업체들 이야기다 년과 년 실적...
4,eco,이제 오를 푸는 일이 남았습니다 학자로서 양심을 걸고 모다모다 샴푸를 보는 안 좋은...
5,eco,기사내용 요약 인사이트 분석결과파운드리 제외 기준 서울뉴시스 이인준 기자 파운드리...
6,eco,세종뉴스 권준 기자 신종 코로나바이러스 감염증코로나의 여파로 사 곳곳에서 부작용이...
7,eco,세종뉴스 서미선 기자 한국경제에 인플레이션 경고등이 켜지면서 국내외 주요 경제예측...
8,eco,기사내용 요약 통계청 한국의 이보고서 발간 중고년 영어 기초학력 미달 배 이상 ...
9,eco,코로나 이 중고등학생의 주요 과목 기초학력 미달 비율이 크게 증가하는 등 학습결손...


In [22]:
stw = pd.read_csv('stopwords-ko.txt', header=None)       # 불용어 제외
stopwords = list(stw.iloc[:, 0].values)
stopwords

def remove_stopwords(doc):
    text = [w for w in doc.split() if w not in stopwords]
    return ' '.join(text)

In [24]:
news_df['news'] = news_df.news.apply(remove_stopwords)

In [25]:
news_df

,cate,news
0,eco,라디오 뉴스슬기로운 라디오생 방송일시 년월 수요일 진이웅 아나운서 출연 박합수 건국...
1,eco,손보 부실 여파 생명 매각도 빨간불실적 개선위 공적자금 투입 차례 매각 시도 번번이...
2,eco,인건비임대료농산물 급등 식음료 기업들 가격 인상 조짐서울 마포구에 거주하는 직장인 ...
3,eco,만대 덜 팔고 조천억원 더 벌었다전 세계 대표 완성차 업체들 이야기다 년과 실적영업...
4,eco,이제 오를 푸는 일이 남았습니다 학자로서 양심을 걸고 모다모다 샴푸를 보는 안 좋은...
5,eco,기사내용 요약 인사이트 분석결과파운드리 제외 기준 서울뉴시스 이인준 기자 파운드리위...
6,eco,세종뉴스 권준 기자 신종 코로나바이러스 감염증코로나의 여파로 곳곳에서 부작용이 발생...
7,eco,세종뉴스 서미선 기자 한국경제에 인플레이션 경고등이 켜지면서 국내외 주요 경제예측 ...
8,eco,기사내용 요약 통계청 한국의 이보고서 발간 중고년 영어 기초학력 미달 배 늘어 폐기...
9,eco,코로나 중고등학생의 주요 과목 기초학력 미달 비율이 크게 증가하는 학습결손이 실된 ...


In [27]:
from konlpy.tag import Hannanum, Komoran, Kkma
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN

In [28]:
kor = Komoran()

In [29]:
news_df.loc[15, 'news']

'이른바 채널 사건 연루 의을 받고 있는 한동 검사장사법연수원 부원장에 대 검찰 수사팀이 무의 처분 계을 보고다는 사실이 알려졌으나 최강욱 더불어민주당 의원은 한 검사장 대전 얘기는 없나며 문제를 제기다윤석열 대통령 당선인의 측근 수사이기 결론이 정진 아니냐는 주장이다 무의 결론이 났으나 윗선에서 수사 종결을 막고 있다는 식의 내용을 언론에 리고 있다는 것이 그의 생각이다최 의원은 라디오 김종배의 시선집중과 인터뷰에서 그대폰 수사가 진되고 있고 증거들이 수집되지 않아서 내지 증거가 나와서 결론으로 될 같다는 식의 얘기가 있어야 한다고 말다 그래야 그나마 논리적으로 뭔가 이뤄지고 있고 순서에 일을 짚어보는 모양이구나 얘기할 수 있다고 다그는 아무런 사정 변경이 없는데 무의 처분하겠다고 보고하는 건 결론을 원하는 측에서 리는 얘기일 수 있다고 주장다 지난 국정감사 검찰이 대전 분석은 진할 것이고 진 중이다 구체적인 것은 말씀 드릴 수 없다고 밝 진전된 게 없다는 이유에서다 그는 수사팀이 무의 종결 보고를 다는 언론 보도조차 사실인지 인야 한다고 의심다채널 사건이란 이동재 전 채널 기자가 이철 전 밸류인베스트코리아 대표에게 여권 인사의 비리를 제보하도록 강요한 사건이다 한 검사장은 이에 연루됐다는 의을 받고 고발됐다 이른바 검언유착 사건으로 불렸다 수사팀은 고발된 전 기자를 강요미수 의로 구속기소으나 한 검사장에 대한 처분은 유보다수사팀은 증거를 찾지 못 무의 처분하겠다는 취지로 상부에 보고다 이성윤 당시 서울중앙지검장은 한 검사장 대폰을 포렌식할 필요가 있다며 반려다 전 기자는 심에서 무죄 판결을 받았다그리고 전날 한 검사장 사건을 수사 중인 서울중앙지검 사부부장 이선가 수사 상을 지검장에게 공식 보고다는 보도가 나왔다 보고서에는 증거 불충분으로 한 검사장의 무의 처분 계을 담은 것으로 알려졌다앞서도 수사팀이 최근 한 검사장 무의 보고를 는데 이정수 서울중앙지검장이 일주일만 기다려 보자고 다는 의이 제기됐다 서울중앙지검은 이에 대 지검장까지 정식 보고되지 않은 상

In [30]:
docs = []

for n in news_df['news']:
    docs.append(kor.nouns(n))

In [31]:
docs

[['라디오',
  '뉴스',
  '슬기',
  '라디오',
  '방송',
  '일시',
  '년',
  '월',
  '수요일',
  '이웅',
  '아나운서',
  '출연',
  '박',
  '합수',
  '건국대',
  '부동산',
  '대학원',
  '겸임',
  '교수',
  '아래',
  '텍스트',
  '실제',
  '방송',
  '내용',
  '차이',
  '수',
  '내용',
  '방송',
  '이웅',
  '아나운서',
  '이하',
  '이웅',
  '슬기',
  '라디오',
  '이슈',
  '인터뷰',
  '시간',
  '준비',
  '대통령 선거',
  '직',
  '서울',
  '아파트',
  '평균',
  '집값',
  '급등',
  '부동산',
  '시장',
  '르',
  '건국대',
  '부동산',
  '대학원',
  '박',
  '합수',
  '겸임',
  '교수',
  '연결',
  '이야기',
  '교수',
  '안녕하세요',
  '박',
  '합수',
  '건국대',
  '부동산',
  '대학원',
  '겸임',
  '교수',
  '이하',
  '박',
  '합수',
  '안녕하세요',
  '이웅',
  '정점',
  '평가',
  '대통령 선거',
  '직',
  '수',
  '가지',
  '평가',
  '움직임',
  '박',
  '합수',
  '자료',
  '더불어민주당',
  '김',
  '의원',
  '대선',
  '직인',
  '일',
  '일',
  '서울',
  '아파트',
  '거래',
  '자료',
  '고가',
  '경신',
  '것',
  '전체',
  '거래',
  '건',
  '이웅',
  '서울',
  '기준',
  '거',
  '박',
  '합수',
  '서울',
  '아파트',
  '거래',
  '건',
  '중',
  '건',
  '정도',
  '고가',
  '갱신',
  '게',
  '정도',
  '얘기',
  '중',
  '상위',
  '개',
  '아파트',
  '원',
  '정도'

In [32]:
for i in range(len(docs)):
    docs[i] = ' '.join(docs[i])

In [33]:
docs

['라디오 뉴스 슬기 라디오 방송 일시 년 월 수요일 이웅 아나운서 출연 박 합수 건국대 부동산 대학원 겸임 교수 아래 텍스트 실제 방송 내용 차이 수 내용 방송 이웅 아나운서 이하 이웅 슬기 라디오 이슈 인터뷰 시간 준비 대통령 선거 직 서울 아파트 평균 집값 급등 부동산 시장 르 건국대 부동산 대학원 박 합수 겸임 교수 연결 이야기 교수 안녕하세요 박 합수 건국대 부동산 대학원 겸임 교수 이하 박 합수 안녕하세요 이웅 정점 평가 대통령 선거 직 수 가지 평가 움직임 박 합수 자료 더불어민주당 김 의원 대선 직인 일 일 서울 아파트 거래 자료 고가 경신 것 전체 거래 건 이웅 서울 기준 거 박 합수 서울 아파트 거래 건 중 건 정도 고가 갱신 게 정도 얘기 중 상위 개 아파트 원 정도 거 상위 개 중 군데 강남 재건축 아파트 얘기 이웅 거래 고가 기록 곳 대부분 강남 쪽 박 합수 재건축 아파트 기대감 작용 재건축 이웅 서울 일반 집값 거래 발 가요 박 합수 전체 성 상태 수 전반 시장 정부 부동산 공약 기대감 일부 매수 심리 지역 일정 부분 복 양상 수 이웅 일부 아파트 집값 세 양극 평가 박 합수 재건축 개발 전면 규제 완 수 지역 위주 기대감 일반 아파트 보합세 이웅 서울 지역 가요 박 합수 전반 수도권 분당 일산 평촌 중동 산본 기 신도시 재건축 리 모델링 기대감 전체 가격 상승 모양새 개 광역시 중 입주 물량 부담 대구 대전 데 약보합 세 소폭 상승 지방 중 강원도 전북 경상남도 등 중심 소폭 상승세 유지 상 이웅 상 전국 박 합수 전반 강보 합 추세 게 이웅 대통령 집무실 이전 관련 용산 쪽 부동산 상승 거래 박 합수 용산 자체 대통령 집무실 이전 서울 중심 지역 특 미군 이전 집중 개발 예고 지역 남 뉴타운 이촌동 재건축 지역 한강로 개발 사업 집중 추진 중 서울역 뒤편 서계 청 파 원로 국제 업무 직 용산 마지막 개발 축 시작 용 트림 시작 지역 집무실 이전 주변 개발 사업 속도 기대 가미 전체 분위기 상 수 이웅 기존 청와대 곳도 개방 기대감 

In [34]:
vec = CountVectorizer(max_df=0.7, min_df=2)

x = vec.fit_transform(docs)
x

<30x783 sparse matrix of type '<class 'numpy.int64'>'
	with 2765 stored elements in Compressed Sparse Row format>

In [35]:
news_arr = x.toarray()
news_arr

array([[10,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  0,  0,  0],
       [ 9,  1,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [37]:
km = KMeans(n_clusters=3)
dbs = DBSCAN(eps=50, min_samples=5)
agg = AgglomerativeClustering(n_clusters=3, linkage='ward')

In [38]:
km.fit_predict(news_arr)

array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       2, 2, 1, 2, 2, 0, 0, 2])

In [39]:
dbs.fit_predict(news_arr)

array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0], dtype=int64)

In [40]:
agg.fit_predict(news_arr)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 0, 0, 2, 2, 0], dtype=int64)

In [ ]:
# n_gram
# original => "뉴스 슬기 로우 라디오 생활 방송 일시"
# 2_gram => 뉴스 슬기 로우 라디오 생활 방송 일시 (뉴스 슬기) (슬기 로우) (로우 라디오) ...
# 띄어쓰기가 있는 단어들(황소 개구리, 스케이트 보드 등)

In [41]:
vec = TfidfVectorizer(max_df=0.7, min_df=2, ngram_range=(1, 2))

x = vec.fit_transform(docs)
x.toarray()

array([[0.18899521, 0.02020364, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09576508, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.50506854, 0.05999109, 0.06473047, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [42]:
km = KMeans(n_clusters=3)
dbs = DBSCAN(eps=50, min_samples=5)
agg = AgglomerativeClustering(n_clusters=3, linkage='ward')

In [43]:
km.fit_predict(news_arr)

array([2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 0, 0])

In [44]:
dbs.fit_predict(news_arr)

array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0], dtype=int64)

In [45]:
agg.fit_predict(news_arr)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 0, 0, 2, 2, 0], dtype=int64)